In [1]:
import krpc
import math
from time import sleep
import PID
from threading import Thread

In [2]:
conn = krpc.connect()

In [3]:
#### Static Variable Definitions

vessel = conn.space_center.active_vessel
kerbin = conn.space_center.bodies['Kerbin']

# Reference frame relative to vessel, with z oriented towards center of kerbin
cframe = conn.space_center.ReferenceFrame.create_hybrid(kerbin.reference_frame,vessel.surface_reference_frame)

# Kerbin Gravitational Parameter
# Requires connection
GM = kerbin.gravitational_parameter #3.5316000e12

In [4]:
# Returns acceleration due to gravity
# Requires connection
def agrav(dist):
    return GM / (dist ** 2)

In [5]:
def VSpeedPID(thrust):

    p = PID.PID()
    p.setPoint(TargetVerticalSpeed)
    
    while TerminateVSpeedPID == False:
        if p.set_point != TargetVerticalSpeed:
            p.setPoint(TargetVerticalSpeed)
        error = p.update(vessel.flight(cframe).vertical_speed)
        #print error
        thrust += (error * vessel.mass) / vessel.max_thrust
        vessel.control.throttle = thrust
        #print thrust
        if vessel.situation == prelaunch:
            break
        sleep(.25)

In [76]:
def HSpeedPID():
    p = PID.PID()
    p.setPoint(TargetHorizontalSpeed)
    
    while TerminateHSpeedPID == False:
        if p.set_point != TargetHorizontalSpeed:
            p.setPoint(TargetHorizontalSpeed)
        # Should probably be distance over speed... 
        # Or maybe a second PID using this one to control distance?
        error = p.update(vessel.flight(cframe).horizontal_speed)
        # Pitch = ArcTan( Vertical Speed / Horizonal Speed )
        vessel.auto_pilot.target_pitch = math.degrees(math.atan(TargetVerticalSpeed/TargetHorizontalSpeed))
        if vessel.situation == prelaunch:
            break
        sleep(.25)
        

In [99]:
def HDirPID():
    p = PID.PID()
    p.setPoint(TargetHeading)
    heading = TargetHeading
    
    while TerminateHDirPID == False:
        if p.set_point != TargetHeading:
            p.setPoint(TargetHeading)
            heading = TargetHeading
        error = p.update(CalcHeading(vessel.flight(cframe).velocity))
        heading -= error
        vessel.auto_pilot.target_heading = heading
        if vessel.situation == prelaunch:
            break
        sleep(3)

In [48]:
# Takes a velocity vector in cframe
# Returns compass heading of horizontal component of vessel velocity
def CalcHeading(VelocityVector):
    Z,Y,X = VelocityVector 
    
    # SW Quadrant
    if X < 0 and Y < 0:
        return 180 + math.degrees(math.atan(abs(X)/abs(Y)))
    # NW Quadrant
    if X < 0 and Y > 0:
        return 360 - math.degrees(math.atan(abs(X)/abs(Y)))
    # NE Quarant
    if X > 0 and Y > 0:
        return math.degrees(math.atan(abs(X)/abs(Y)))
    # SE Quadrant
    if X > 0 and Y < 0:
        return 180 - math.degrees(math.atan(abs(X)/abs(Y)))
    # Corner Cases (to avoid zero division error)
    if X == 0 and Y > 0:
        return 0
    if X == 0 and Y < 0:
        return 180
    if X > 0 and Y == 0:
        return 90
    if X < 0 and Y == 0:
        return 270

### Execution

In [100]:
prelaunch = vessel.situation

vessel.control.sas = False
vessel.control.rcs = False
vessel.auto_pilot.engage()
vessel.auto_pilot.target_pitch_and_heading(90, 90)
vessel.auto_pilot.target_roll = 0

vessel.control.throttle = 1.0
vessel.control.activate_next_stage()
sleep(1)
vessel.control.throttle = 0.0

# Engine off until near 0 vertical velocity
while True:
    if vessel.flight(cframe).vertical_speed < .001:
        break

thrust = ((agrav(vessel.position(cframe)[0]) * vessel.mass) / vessel.max_thrust)
vessel.control.throttle = thrust

In [101]:
TargetVerticalSpeed = 0.0
TerminateVSpeedPID = False
ThrottleThread = Thread(target = VSpeedPID, args = (thrust,))
ThrottleThread.start()


TargetHeading = 90
TerminateHDirPID = False
HeadingThread = Thread(target = HDirPID)
HeadingThread.start()


#TerminateHSpeedPID = False
#vessel.auto_pilot.target_heading = 90

In [102]:
vessel.auto_pilot.target_pitch = 8
TargetHeading = 270

In [90]:
CalcHeading(vessel.flight(cframe).velocity)

12.059847358551824

In [70]:
vessel.flight(cframe).velocity

(0.005216584138513147, 2.999907571422095, -5.55066303291548)

In [39]:
start = vessel.position(kerbin.reference_frame)

In [87]:
CalcHeading(conn.space_center.transform_position(start,kerbin.reference_frame, cframe))

182.45807424740588

In [75]:
vessel.auto_pilot.target_heading

-476.2026672363281

Exception in thread Thread-19:
Traceback (most recent call last):
  File "C:\Users\Will\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\Will\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-5-a8a6b6d89703>", line 11, in VSpeedPID
    thrust += (error * vessel.mass) / vessel.max_thrust
ZeroDivisionError: float division by zero

